<a href="https://colab.research.google.com/github/softstat/analysis/blob/main/Angle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [91]:
cd drive

[Errno 2] No such file or directory: 'drive'
/content/drive/MyDrive/Data


In [92]:
cd MyDrive

[Errno 2] No such file or directory: 'MyDrive'
/content/drive/MyDrive/Data


In [93]:
cd Data

[Errno 2] No such file or directory: 'Data'
/content/drive/MyDrive/Data


In [94]:
ls

 angle_gait.csv              data.csv            weather.csv
'Clinical Assessment.xlsx'   titanic_train.csv


In [95]:
import pandas as pd
angle = pd.read_csv('angle_gait.csv')
clinical_assessment = pd.read_excel('Clinical Assessment.xlsx')

In [96]:
angle.drop('Unnamed: 0',axis=1,inplace=True)

In [97]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
import warnings
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score, classification_report

In [98]:
warnings.filterwarnings(action='ignore')

In [99]:
def perform_pca(data, num_components):
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(data)
    pca = PCA(n_components=num_components)
    pca_result = pca.fit_transform(scaled_data)
    return pca_result

In [100]:
col = angle.columns
col = col.str.replace('[^\w\s]', '', regex=True)
angle.columns = col
num_components = 43

In [101]:
angle = angle.dropna()
na_sum = angle.isna().sum()

In [102]:
import random
random.seed(1)
total_angle_gait_pca = angle.copy()
total_angle_model = PCA(n_components=num_components)
total_angle_result = total_angle_model.fit_transform(total_angle_gait_pca)

In [103]:
total_angle_result

array([[-1.69715644e+02,  2.78757824e+01,  1.24143094e+02, ...,
        -3.92471603e+00,  2.51926619e+00,  8.50845056e-14],
       [-2.23333375e+02, -1.54750144e+01,  4.69437877e+01, ...,
        -4.22573239e-01,  3.29514125e+00,  8.50845056e-14],
       [ 6.22312161e+01, -1.43502742e+01,  5.24627212e+02, ...,
        -6.68759815e-01,  2.06298071e+00,  8.50845056e-14],
       ...,
       [ 2.16404983e+02,  2.36416445e+02, -8.90716272e+01, ...,
         2.55564438e-02,  7.50927346e+00,  8.50845056e-14],
       [-1.13801129e+01, -3.60015911e+01, -4.82583542e+01, ...,
         1.01146215e+01,  5.30972112e+00,  8.50845056e-14],
       [ 3.27822467e+01, -1.90162479e+02,  1.36776906e+02, ...,
         4.80209806e+00,  6.26140866e+00,  8.50845056e-14]])

In [104]:
import numpy as np
angle = np.array(angle)
non_angle = angle[:,0:2424]

In [105]:
random.seed(123)
non_angle_model = PCA(n_components=num_components)
non_angle_result = non_angle_model.fit_transform(non_angle)

In [106]:
non_angle_result.shape

(43, 43)

In [107]:
par_angle = angle[:,2525:4848]
random.seed(123)
par_angle_model = PCA(n_components=num_components)
par_angle_result = par_angle_model.fit_transform(par_angle)

In [108]:
score = clinical_assessment.iloc[:,9]

In [109]:
score = score.dropna()
del score[0]
del score[15]

In [110]:
y = []
for value in score:
    if value >= 45:
        y.append(1)
    else:
        y.append(0)
print(y)

[0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0]


## Non-paretic Side

In [111]:
df_non = pd.DataFrame(non_angle_result)

In [112]:
y = pd.DataFrame(y)

In [113]:
non = pd.concat([y,df_non],axis=1)

In [114]:
non.columns = ['PC'+ str(i) for i in range(0,44)]

In [115]:
colname = list(non.columns)
colname[0] = "score"
non.columns = colname
non.columns

Index(['score', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9',
       'PC10', 'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16', 'PC17', 'PC18',
       'PC19', 'PC20', 'PC21', 'PC22', 'PC23', 'PC24', 'PC25', 'PC26', 'PC27',
       'PC28', 'PC29', 'PC30', 'PC31', 'PC32', 'PC33', 'PC34', 'PC35', 'PC36',
       'PC37', 'PC38', 'PC39', 'PC40', 'PC41', 'PC42', 'PC43'],
      dtype='object')

In [116]:
X = non.loc[:,'PC1':'PC43']
y = non['score']
X_train , X_test , y_train , y_test  = train_test_split(X,y,test_size = 0.4,random_state=123)

### logistic regression
- f1-score : 0.777

In [117]:
from sklearn.feature_selection import RFECV
lr_model = LogisticRegression()

# RFECV를 사용하여 최적의 특성 선택
rfecv = RFECV(estimator=lr_model, cv=5)
X_train_rfecv = rfecv.fit_transform(X_train, y_train)

# 선택된 특성으로 모델 학습
lr_model.fit(X_train_rfecv, y_train)

# 테스트 데이터에도 선택된 특성을 적용
X_test_rfecv = rfecv.transform(X_test)

# 테스트 데이터에 대한 예측
y_pred_lr = lr_model.predict(X_test_rfecv)

conf_matrix = confusion_matrix(y_test, y_pred_lr)
print("Confusion Matrix:\n", conf_matrix)
# 정확도 계산
accuracy_lr = accuracy_score(y_test, y_pred_lr)
print("Test Accuracy:", accuracy_lr)

# f1-score 계산
f1_lr = f1_score(y_test, y_pred_lr)
print("f1-score:", f1_lr)

# auc 계산
auc_lr = roc_auc_score(y_test,y_pred_lr)
print("AUC:",auc_lr)

# 분류 보고서 출력 (정밀도, 재현율, F1 점수 등)
classification_rep_lr = classification_report(y_test, y_pred_lr)
print("Classification Report:\n", classification_rep_lr)

# 최적의 특성 개수 출력
print("Optimal Number of Features:", rfecv.n_features_)

Confusion Matrix:
 [[7 0]
 [4 7]]
Test Accuracy: 0.7777777777777778
f1-score: 0.7777777777777778
AUC: 0.8181818181818181
Classification Report:
               precision    recall  f1-score   support

           0       0.64      1.00      0.78         7
           1       1.00      0.64      0.78        11

    accuracy                           0.78        18
   macro avg       0.82      0.82      0.78        18
weighted avg       0.86      0.78      0.78        18

Optimal Number of Features: 6


### Random Forest
- f1-score : 0.846

In [118]:
from sklearn.feature_selection import RFECV
rf_model = RandomForestClassifier(n_estimators=100,random_state=123)

# RFECV를 사용하여 최적의 특성 선택
rfecv = RFECV(estimator=rf_model, cv=5)
X_train_rfecv = rfecv.fit_transform(X_train, y_train)

# 선택된 특성으로 모델 학습
rf_model.fit(X_train_rfecv, y_train)

# 테스트 데이터에도 선택된 특성을 적용
X_test_rfecv = rfecv.transform(X_test)

# 테스트 데이터에 대한 예측
y_pred_rf = rf_model.predict(X_test_rfecv)

conf_matrix = confusion_matrix(y_test, y_pred_rf)
print("Confusion Matrix:\n", conf_matrix)

# 정확도 계산
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Test Accuracy:", accuracy_rf)

# f1-score 계산
f1_rf = f1_score(y_test, y_pred_rf)
print("f1-score:", f1_rf)

# AUC 계산
auc_rf = roc_auc_score(y_test,y_pred_rf)
print("AUC:",auc_rf)


# 분류 보고서 출력 (정밀도, 재현율, F1 점수 등)
classification_rep_rf = classification_report(y_test, y_pred_rf)
print("Classification Report:\n", classification_rep_rf)

# 최적의 특성 개수 출력
print("Optimal Number of Features:", rfecv.n_features_)

Confusion Matrix:
 [[ 3  4]
 [ 0 11]]
Test Accuracy: 0.7777777777777778
f1-score: 0.846153846153846
AUC: 0.7142857142857143
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.43      0.60         7
           1       0.73      1.00      0.85        11

    accuracy                           0.78        18
   macro avg       0.87      0.71      0.72        18
weighted avg       0.84      0.78      0.75        18

Optimal Number of Features: 5


### SVM
- f1-score : 0.777

In [119]:
from sklearn.feature_selection import RFECV
svm_model = SVC(kernel='linear',random_state=123)

# RFECV를 사용하여 최적의 특성 선택
rfecv = RFECV(estimator=svm_model, cv=5)
X_train_rfecv = rfecv.fit_transform(X_train, y_train)

# 선택된 특성으로 모델 학습
svm_model.fit(X_train_rfecv, y_train)

# 테스트 데이터에도 선택된 특성을 적용
X_test_rfecv = rfecv.transform(X_test)

# 테스트 데이터에 대한 예측
y_pred_svm = svm_model.predict(X_test_rfecv)

conf_matrix = confusion_matrix(y_test, y_pred_svm)
print("Confusion Matrix:\n", conf_matrix)

# 정확도 계산
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print("Test Accuracy:", accuracy_svm)

# f1-score 계산
f1_svm = f1_score(y_test, y_pred_svm)
print("f1-score:", f1_svm)

# AUC 계산
auc_svm = roc_auc_score(y_test,y_pred_svm)
print("AUC:",auc_svm)

# 분류 보고서 출력 (정밀도, 재현율, F1 점수 등)
classification_rep_svm = classification_report(y_test, y_pred_svm)
print("Classification Report:\n", classification_rep_svm)

# 최적의 특성 개수 출력
print("Optimal Number of Features:", rfecv.n_features_)

Confusion Matrix:
 [[7 0]
 [4 7]]
Test Accuracy: 0.7777777777777778
f1-score: 0.7777777777777778
AUC: 0.8181818181818181
Classification Report:
               precision    recall  f1-score   support

           0       0.64      1.00      0.78         7
           1       1.00      0.64      0.78        11

    accuracy                           0.78        18
   macro avg       0.82      0.82      0.78        18
weighted avg       0.86      0.78      0.78        18

Optimal Number of Features: 5


### XGBoost
- f1-score: 0.667

In [120]:
from sklearn.feature_selection import RFECV
import xgboost as xgb
xgb_model = xgb.XGBClassifier(random_state=123)

# RFECV를 사용하여 최적의 특성 선택
rfecv = RFECV(estimator=xgb_model, cv=5)
X_train_rfecv = rfecv.fit_transform(X_train, y_train)

# 선택된 특성으로 모델 학습
xgb_model.fit(X_train_rfecv, y_train)

# 테스트 데이터에도 선택된 특성을 적용
X_test_rfecv = rfecv.transform(X_test)

# 테스트 데이터에 대한 예측
y_pred_xgb = xgb_model.predict(X_test_rfecv)

conf_matrix = confusion_matrix(y_test, y_pred_xgb)
print("Confusion Matrix:\n", conf_matrix)

# 정확도 계산
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print("Test Accuracy:", accuracy_xgb)

# f1-score 계산
f1_xgb = f1_score(y_test, y_pred_xgb)
print("f1-score:", f1_xgb)

# AUC 계산
auc_xgb = roc_auc_score(y_test,y_pred_xgb)
print("AUC:",auc_xgb)

# 분류 보고서 출력 (정밀도, 재현율, F1 점수 등)
classification_rep_xgb = classification_report(y_test, y_pred_xgb)
print("Classification Report:\n", classification_rep_xgb)

# 최적의 특성 개수 출력
print("Optimal Number of Features:", rfecv.n_features_)

Confusion Matrix:
 [[6 1]
 [5 6]]
Test Accuracy: 0.6666666666666666
f1-score: 0.6666666666666665
AUC: 0.7012987012987013
Classification Report:
               precision    recall  f1-score   support

           0       0.55      0.86      0.67         7
           1       0.86      0.55      0.67        11

    accuracy                           0.67        18
   macro avg       0.70      0.70      0.67        18
weighted avg       0.74      0.67      0.67        18

Optimal Number of Features: 1


### LDA
- f1-score : 0.692

In [121]:
# LDA 모델
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda_model = LinearDiscriminantAnalysis()
# RFECV를 사용하여 최적의 특성 선택
rfecv = RFECV(estimator=lda_model, cv=5)
X_train_rfecv = rfecv.fit_transform(X_train, y_train)

# 선택된 특성으로 모델 학습
lda_model.fit(X_train_rfecv, y_train)

# 테스트 데이터에도 선택된 특성을 적용
X_test_rfecv = rfecv.transform(X_test)

# 테스트 데이터에 대한 예측
y_pred_lda = lda_model.predict(X_test_rfecv)

conf_matrix = confusion_matrix(y_test, y_pred_lda)
print("Confusion Matrix:\n", conf_matrix)

# 정확도 계산
accuracy_lda = accuracy_score(y_test, y_pred_lda)
print("Test Accuracy:", accuracy_lda)

# f1-score 계산
f1_lda = f1_score(y_test, y_pred_lda)
print("f1-score:", f1_lda)

# AUC 계산
auc_lda = roc_auc_score(y_test,y_pred_lda)
print("AUC:",auc_lda)

# 분류 보고서 출력 (정밀도, 재현율, F1 점수 등)
classification_rep_lda = classification_report(y_test, y_pred_lda)
print("Classification Report:\n", classification_rep_lda)

# 최적의 특성 개수 출력
print("Optimal Number of Features:", rfecv.n_features_)

Confusion Matrix:
 [[1 6]
 [2 9]]
Test Accuracy: 0.5555555555555556
f1-score: 0.6923076923076923
AUC: 0.48051948051948057
Classification Report:
               precision    recall  f1-score   support

           0       0.33      0.14      0.20         7
           1       0.60      0.82      0.69        11

    accuracy                           0.56        18
   macro avg       0.47      0.48      0.45        18
weighted avg       0.50      0.56      0.50        18

Optimal Number of Features: 2


## paretic-Side

In [122]:
df_par = pd.DataFrame(par_angle_result)
y = pd.DataFrame(y)
par = pd.concat([y,df_par],axis=1)
par.columns = ['PC'+ str(i) for i in range(0,44)]
colname = list(par.columns)
colname[0] = "score"
par.columns = colname
par.columns

Index(['score', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9',
       'PC10', 'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16', 'PC17', 'PC18',
       'PC19', 'PC20', 'PC21', 'PC22', 'PC23', 'PC24', 'PC25', 'PC26', 'PC27',
       'PC28', 'PC29', 'PC30', 'PC31', 'PC32', 'PC33', 'PC34', 'PC35', 'PC36',
       'PC37', 'PC38', 'PC39', 'PC40', 'PC41', 'PC42', 'PC43'],
      dtype='object')

In [123]:
random.seed(1)
X = par.loc[:,'PC1':'PC30']
y = par['score']
X_train , X_test , y_train , y_test  = train_test_split(X,y,test_size = 0.4,random_state=123)

In [124]:
k_best = SelectKBest(f_classif, k=20)  # 예제로 20개의 최상위 특성 선택
X_train_selected = k_best.fit_transform(X_train, y_train)
X_test_selected = k_best.transform(X_test)

### logistic regression
- f1-score : 0.583

In [125]:
from sklearn.feature_selection import RFECV
lr_model = LogisticRegression()

# RFECV를 사용하여 최적의 특성 선택
rfecv = RFECV(estimator=lr_model, cv=5)
X_train_rfecv = rfecv.fit_transform(X_train, y_train)

# 선택된 특성으로 모델 학습
lr_model.fit(X_train_rfecv, y_train)

# 테스트 데이터에도 선택된 특성을 적용
X_test_rfecv = rfecv.transform(X_test)

# 테스트 데이터에 대한 예측
y_pred_lr = lr_model.predict(X_test_rfecv)

conf_matrix = confusion_matrix(y_test, y_pred_lr)
print("Confusion Matrix:\n", conf_matrix)
# 정확도 계산
accuracy_lr_ps = accuracy_score(y_test, y_pred_lr)
print("Test Accuracy:", accuracy_lr_ps)

# f1-score 계산
f1_lr_ps = f1_score(y_test, y_pred_lr)
print("f1-score:", f1_lr_ps)

# AUC 계산
auc_lr_ps = roc_auc_score(y_test,y_pred_lr)
print("AUC:",auc_lr_ps)

# 분류 보고서 출력 (정밀도, 재현율, F1 점수 등)
classification_rep_lr = classification_report(y_test, y_pred_lr)
print("Classification Report:\n", classification_rep_lr)

# 최적의 특성 개수 출력
print("Optimal Number of Features:", rfecv.n_features_)

Confusion Matrix:
 [[1 6]
 [4 7]]
Test Accuracy: 0.4444444444444444
f1-score: 0.5833333333333334
AUC: 0.38961038961038963
Classification Report:
               precision    recall  f1-score   support

           0       0.20      0.14      0.17         7
           1       0.54      0.64      0.58        11

    accuracy                           0.44        18
   macro avg       0.37      0.39      0.38        18
weighted avg       0.41      0.44      0.42        18

Optimal Number of Features: 2


### paretic RandomForest
- f1-score : 0.786

In [126]:
from sklearn.feature_selection import RFECV
rf_model = RandomForestClassifier(n_estimators=100,random_state=123)

# RFECV를 사용하여 최적의 특성 선택
rfecv = RFECV(estimator=rf_model, cv=5)
X_train_rfecv = rfecv.fit_transform(X_train, y_train)

# 선택된 특성으로 모델 학습
rf_model.fit(X_train_rfecv, y_train)

# 테스트 데이터에도 선택된 특성을 적용
X_test_rfecv = rfecv.transform(X_test)

# 테스트 데이터에 대한 예측
y_pred_rf = rf_model.predict(X_test_rfecv)

conf_matrix = confusion_matrix(y_test, y_pred_rf)
print("Confusion Matrix:\n", conf_matrix)

# 정확도 계산
accuracy_rf_ps = accuracy_score(y_test, y_pred_rf)
print("Test Accuracy:", accuracy_rf_ps)

# f1-score 계산
f1_rf_ps = f1_score(y_test, y_pred_rf)
print("f1-score:", f1_rf_ps)

# AUC 계산
auc_rf_ps = roc_auc_score(y_test,y_pred_rf)
print("AUC:",auc_rf_ps)


# 분류 보고서 출력 (정밀도, 재현율, F1 점수 등)
classification_rep_rf = classification_report(y_test, y_pred_rf)
print("Classification Report:\n", classification_rep_rf)

# 최적의 특성 개수 출력
print("Optimal Number of Features:", rfecv.n_features_)

Confusion Matrix:
 [[ 1  6]
 [ 0 11]]
Test Accuracy: 0.6666666666666666
f1-score: 0.7857142857142858
AUC: 0.5714285714285714
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.14      0.25         7
           1       0.65      1.00      0.79        11

    accuracy                           0.67        18
   macro avg       0.82      0.57      0.52        18
weighted avg       0.78      0.67      0.58        18

Optimal Number of Features: 6


### paretic SVM
- f1-score : 0.5217

In [127]:
from sklearn.feature_selection import RFECV
svm_model = SVC(kernel='linear',random_state=123)

# RFECV를 사용하여 최적의 특성 선택
rfecv = RFECV(estimator=svm_model, cv=5)
X_train_rfecv = rfecv.fit_transform(X_train, y_train)

# 선택된 특성으로 모델 학습
svm_model.fit(X_train_rfecv, y_train)

# 테스트 데이터에도 선택된 특성을 적용
X_test_rfecv = rfecv.transform(X_test)

# 테스트 데이터에 대한 예측
y_pred_svm = svm_model.predict(X_test_rfecv)

conf_matrix = confusion_matrix(y_test, y_pred_svm)
print("Confusion Matrix:\n", conf_matrix)

# 정확도 계산
accuracy_svm_ps = accuracy_score(y_test, y_pred_svm)
print("Test Accuracy:", accuracy_svm_ps)

# f1-score 계산
f1_svm_ps = f1_score(y_test, y_pred_svm)
print("f1-score:", f1_svm_ps)

# AUC 계산
auc_svm_ps = roc_auc_score(y_test,y_pred_svm)
print("AUC:",auc_svm_ps)


# 분류 보고서 출력 (정밀도, 재현율, F1 점수 등)
classification_rep_svm = classification_report(y_test, y_pred_svm)
print("Classification Report:\n", classification_rep_svm)

# 최적의 특성 개수 출력
print("Optimal Number of Features:", rfecv.n_features_)

Confusion Matrix:
 [[1 6]
 [5 6]]
Test Accuracy: 0.3888888888888889
f1-score: 0.5217391304347826
AUC: 0.34415584415584416
Classification Report:
               precision    recall  f1-score   support

           0       0.17      0.14      0.15         7
           1       0.50      0.55      0.52        11

    accuracy                           0.39        18
   macro avg       0.33      0.34      0.34        18
weighted avg       0.37      0.39      0.38        18

Optimal Number of Features: 5


### paretic XGBoost
- f1-score : 0.667

In [128]:
from sklearn.feature_selection import RFECV
import xgboost as xgb
xgb_model = xgb.XGBClassifier(random_state=123)

# RFECV를 사용하여 최적의 특성 선택
rfecv = RFECV(estimator=xgb_model, cv=5)
X_train_rfecv = rfecv.fit_transform(X_train, y_train)

# 선택된 특성으로 모델 학습
xgb_model.fit(X_train_rfecv, y_train)

# 테스트 데이터에도 선택된 특성을 적용
X_test_rfecv = rfecv.transform(X_test)

# 테스트 데이터에 대한 예측
y_pred_xgb = xgb_model.predict(X_test_rfecv)

conf_matrix = confusion_matrix(y_test, y_pred_xgb)
print("Confusion Matrix:\n", conf_matrix)

# 정확도 계산
accuracy_xgb_ps = accuracy_score(y_test, y_pred_xgb)
print("Test Accuracy:", accuracy_xgb_ps)

# f1-score 계산
f1_xgb_ps = f1_score(y_test, y_pred_xgb)
print("f1-score:", f1_xgb_ps)

# AUC 계산
auc_xgb_ps = roc_auc_score(y_test,y_pred_xgb)
print("AUC:",auc_xgb_ps)


# 분류 보고서 출력 (정밀도, 재현율, F1 점수 등)
classification_rep_xgb = classification_report(y_test, y_pred_xgb)
print("Classification Report:\n", classification_rep_xgb)

# 최적의 특성 개수 출력
print("Optimal Number of Features:", rfecv.n_features_)

Confusion Matrix:
 [[2 5]
 [3 8]]
Test Accuracy: 0.5555555555555556
f1-score: 0.6666666666666667
AUC: 0.5064935064935066
Classification Report:
               precision    recall  f1-score   support

           0       0.40      0.29      0.33         7
           1       0.62      0.73      0.67        11

    accuracy                           0.56        18
   macro avg       0.51      0.51      0.50        18
weighted avg       0.53      0.56      0.54        18

Optimal Number of Features: 3


### paretic LDA  
- f1-score : 0.43

In [129]:
# LDA 모델
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda_model = LinearDiscriminantAnalysis()
# RFECV를 사용하여 최적의 특성 선택
rfecv = RFECV(estimator=lda_model, cv=5)
X_train_rfecv = rfecv.fit_transform(X_train, y_train)

# 선택된 특성으로 모델 학습
lda_model.fit(X_train_rfecv, y_train)

# 테스트 데이터에도 선택된 특성을 적용
X_test_rfecv = rfecv.transform(X_test)

# 테스트 데이터에 대한 예측
y_pred_lda = lda_model.predict(X_test_rfecv)

conf_matrix = confusion_matrix(y_test, y_pred_lda)
print("Confusion Matrix:\n", conf_matrix)

# 정확도 계산
accuracy_lda_ps = accuracy_score(y_test, y_pred_lda)
print("Test Accuracy:", accuracy_lda_ps)

# f1-score 계산
f1_lda_ps = f1_score(y_test, y_pred_lda)
print("f1-score:", f1_lda_ps)

# AUC 계산
auc_lda_ps = roc_auc_score(y_test,y_pred_lda)
print("AUC:",auc_lda_ps)

# 분류 보고서 출력 (정밀도, 재현율, F1 점수 등)
classification_rep_lda = classification_report(y_test, y_pred_lda)
print("Classification Report:\n", classification_rep_lda)

# 최적의 특성 개수 출력
print("Optimal Number of Features:", rfecv.n_features_)

Confusion Matrix:
 [[0 7]
 [6 5]]
Test Accuracy: 0.2777777777777778
f1-score: 0.43478260869565216
AUC: 0.22727272727272727
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.42      0.45      0.43        11

    accuracy                           0.28        18
   macro avg       0.21      0.23      0.22        18
weighted avg       0.25      0.28      0.27        18

Optimal Number of Features: 8


In [130]:
### AUC 결과
result = {'model':["logistic","RandomForest","SVM","XGBoost","LDA"],
          'f1_score':[f1_lr,f1_rf,f1_svm,f1_xgb,f1_lda],
          'paretic_f1_score':[f1_lr_ps,f1_rf_ps,f1_svm_ps,f1_xgb_ps,f1_lda_ps],
          'AUC':[auc_lr,auc_rf,auc_svm,auc_xgb,auc_lda],
          'Paretic_AUC':[auc_lr_ps,auc_rf_ps,auc_svm_ps,auc_xgb_ps,auc_lda_ps]}


In [131]:
result = pd.DataFrame(result)
result

,model,f1_score,paretic_f1_score,AUC,Paretic_AUC
0,logistic,0.777778,0.583333,0.818182,0.389610
1,RandomForest,0.846154,0.785714,0.714286,0.571429
2,SVM,0.777778,0.521739,0.818182,0.344156
3,XGBoost,0.666667,0.666667,0.701299,0.506494
4,LDA,0.692308,0.434783,0.480519,0.227273


In [132]:
!jupyter nbconvert --to html "/content/drive/MyDrive/Colab Notebooks/Angle.ipynb"

[NbConvertApp] Converting notebook /content/drive/MyDrive/Colab Notebooks/Angle.ipynb to html
[NbConvertApp] Writing 679529 bytes to /content/drive/MyDrive/Colab Notebooks/Angle.html
